In [1]:
import numpy as np
from pyscf import gto, scf, fci


$$
\begin{split}
E_H[\gamma_1] & = \frac{1}{2} \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{a,\kappa} c_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma_1] & = \frac{1}{2} \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_a(r) \phi_b(r')^{\ast}  \phi_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & = \frac{1}{2} \iint \frac{\gamma_1(r,r) \gamma_1(r',r')}{|r-r'|} dr dr' \\
\end{split}
$$

In [2]:
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu,nu,kappa,lamda]

    return 1/2.*energy



$$
\begin{split}
E_x[\gamma_1] & = -\frac{1}{4} \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x[\gamma_1] & = -\frac{1}{4} \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_x[\gamma_1] & = -\frac{1}{4} \iint \frac{\gamma_1(r,r') \gamma_1(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [3]:
def ONERDMFT_exchange_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,b]*C[kappa,b]*C[lamda,a]*Fouridx[mu,nu,kappa,lamda]

    return -1/4.*energy


$$
\begin{split}
E_{xc}[\gamma_1] & = -\frac{1}{4} \sum_a^M \sum_b^M \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{b,\kappa} c_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_{xc}[\gamma_1] & = -\frac{1}{4} \iint \sum_a^M \sum_b^M \sqrt{n_a n_b} \frac{\phi_a(r)^{\ast} \phi_b(r) \phi_b(r')^{\ast}  \phi_a(r')}{|r-r'|} dr dr' \\
E_{xc}[\gamma_1] & = -\frac{1}{4} \iint \frac{\gamma_1^{\frac{1}{2}}(r,r') \gamma_1^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

In [4]:
def ONERDMFT_Mueller_functional(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return -1/4.*energy

In [5]:
def ONERDMFT_Mueller_Full(Fouridx, C, n, mol, h):
    E_h = np.trace(np.matmul(h, np.matmul(np.matmul(C.T,np.diag(n)), C)))
    U = ONERDMFT_hartree_energy(Fouridx, C, n, mol)
    E_xc = ONERDMFT_Mueller_functional(Fouridx, C, n, mol)
    print(E_h, U, E_xc)
    return E_h + U + E_xc

In [6]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')


## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()

N = mol.nelec[0]
gamma=np.matmul(C[:,0:N],C[:,0:N].T)*2


#occu, naturalC = np.linalg.eigh(np.matmul(S, gamma))
occu, naturalC = np.linalg.eigh(gamma)

print(gamma)
print(np.matmul(np.matmul(naturalC.T,np.diag(occu)), naturalC))

converged SCF energy = -128.473876870668
[[ 2.1019495  -0.22714523 -0.27610773  0.          0.          0.
   0.          0.          0.        ]
 [-0.22714523  0.59606287  0.59832762  0.          0.          0.
   0.          0.          0.        ]
 [-0.27610773  0.59832762  0.60174891  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.95287412  0.          0.
   0.63408178  0.          0.        ]
 [ 0.          0.          0.          0.          0.95287412  0.
   0.          0.63408178  0.        ]
 [ 0.          0.          0.          0.          0.          0.95287412
   0.          0.          0.63408178]
 [ 0.          0.          0.          0.63408178  0.          0.
   0.42194419  0.          0.        ]
 [ 0.          0.          0.          0.          0.63408178  0.
   0.          0.42194419  0.        ]
 [ 0.          0.          0.          0.          0.          0.63408178
   0.          0.          0.42194419

In [7]:
# run FCI 
FCI = True
if FCI:
    fs = fci.FCI(mol, mf.mo_coeff)
    e, ci = fs.kernel(verbose=0)
print(f"E FCI: {e}; E HF: {mf.e_tot} E_c {e-mf.e_tot}")

E FCI: -128.5898023498827; E HF: -128.4738768706684 E_c -0.11592547921429741


In [8]:
FCIgamma = fci.direct_spin1.make_rdm1(ci, mf.mo_coeff.shape[0], mol.nelec)
FCIoccu, FCInaturalC = np.linalg.eigh(FCIgamma)

#occu = np.diagonal(np.matmul(np.diag(occu), S))

#print(FCIoccu, occu)

print(sum(FCIoccu), sum(occu), np.trace(FCIgamma), np.trace(gamma))

10.000000000000004 7.424216223676516 10.000000000000007 7.424216223676514


In [9]:
#for i, n  in enumerate(occu):
#    if n < 0:
#        occu[i] = 0 

In [10]:
print(np.trace(np.matmul(h,gamma))+ONERDMFT_hartree_energy(eri, naturalC, occu, mol)+ONERDMFT_exchange_energy(eri, naturalC, occu, mol))
print(np.trace(np.matmul(h,FCIgamma))+ONERDMFT_hartree_energy(eri, FCInaturalC, FCIoccu, mol)+ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))

-128.47387687066853
-134.98409669251782


In [11]:
print("Mueller Energy, Exchange Energy of HF-gamma:", ONERDMFT_Mueller_functional(eri, naturalC, occu, mol),ONERDMFT_exchange_energy(eri, naturalC, occu, mol))
#print("Mueller Correlation Energy:", ONERDMFT_Mueller_functional(eri, FCInaturalC, FCIoccu, mol)-ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))
print("Mueller Energy, Exchange Energy of FCI-gamma:", ONERDMFT_Mueller_functional(eri, FCInaturalC, FCIoccu, mol),ONERDMFT_exchange_energy(eri, FCInaturalC, FCIoccu, mol))


/tmp/ipykernel_979547/3502421573.py:11: RuntimeWarning: invalid value encountered in sqrt
  energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]


Mueller Energy, Exchange Energy of HF-gamma: nan -12.127575803200866
Mueller Energy, Exchange Energy of FCI-gamma: -7.718834779811283 -15.125638160680534


In [12]:
#E_h = np.trace(np.matmul(h, np.matmul(np.matmul(C.T,np.diagonal(n)), C)))

print(f"Hartree Fock Eotal Energy {mf.e_tot}")
print(f"Mueller Total Energy from FCI gamma {ONERDMFT_Mueller_Full(eri, FCInaturalC, FCIoccu, mol, h)}")
print(f"Mueller Total Energy from HF gamma {ONERDMFT_Mueller_Full(eri, naturalC, occu, mol, h)}")


Hartree Fock Eotal Energy -128.4738768706684
-195.11139302626322 75.37825269804213 -7.718834779811283
Mueller Total Energy from FCI gamma -127.45197510803237


/tmp/ipykernel_979547/3502421573.py:11: RuntimeWarning: invalid value encountered in sqrt
  energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]


-125.92042548853155 66.27654288017754 nan
Mueller Total Energy from HF gamma nan
